In [1]:
# =============================================================================
# 1. 安裝區：安裝最新的 Google GenAI SDK 和其他工具
# =============================================================================
print("⏳ 正在安裝最新版 Google GenAI SDK 與其他工具，請稍候 (約 1-2 分鐘)...")
from IPython.utils import io
with io.capture_output() as captured:
    # 注意：這裡我們安裝 'google-genai' 這是最新的官方套件
    !pip install -U -q google-genai openai-whisper gradio transformers torch accelerate requests

print("✅ 工具安裝完成！開始匯入並啟動...")

⏳ 正在安裝最新版 Google GenAI SDK 與其他工具，請稍候 (約 1-2 分鐘)...
✅ 工具安裝完成！開始匯入並啟動...


In [2]:
# =============================================================================
# 2. 匯入區：使用最新的官方語法
# =============================================================================
import os
import time
import requests
from urllib.parse import quote
import gradio as gr
import whisper
from transformers import pipeline

# ❗ 這裡就是你發現的最新官方語法！
from google import genai
from google.genai import types

import warnings
warnings.filterwarnings("ignore")

In [3]:
!apt-get update -y
!apt-get install -y fonts-noto-cjk
!fc-list | grep -i "NotoSansCJK" | head

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly insta

In [4]:
# =============================================================================
# 3. 模型準備區
# =============================================================================
print("⏳ 正在叫醒 Whisper 聽寫機器人...")
whisper_model = whisper.load_model("medium")

emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    return_all_scores=True  # ✅ 回傳多維度分數
)

print("✅ 所有機器人都準備好了！")

⏳ 正在叫醒 Whisper 聽寫機器人...


Device set to use cpu


✅ 所有機器人都準備好了！


In [5]:
from PIL import Image, ImageDraw, ImageFont
import re

FONT_PATH_REG = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
FONT_PATH_BOLD = "/usr/share/fonts/opentype/noto/NotoSansCJK-Bold.ttc"

def wrap_by_pixel(draw, text, font, max_width):
    text = re.sub(r"\s+", " ", text).strip()
    if not text:
        return []
    lines, buf = [], ""
    for ch in text:
        test = buf + ch
        w = draw.textbbox((0, 0), test, font=font)[2]
        if w <= max_width:
            buf = test
        else:
            if buf:
                lines.append(buf)
            buf = ch
    if buf:
        lines.append(buf)
    return lines

def build_basic_prompt(transcribed_text, detected_emotion):
    """
    不依賴 Gemini 的基礎提示詞（穩定、一定能用）
    """
    text = transcribed_text.strip()
    emo = detected_emotion.lower()

    return (
        "Cute 3D Pixar style sticker, chibi character, "
        f"clear facial expression showing {emo} emotion, "
        "simple and clean composition, "
        "white background, high quality, vector style, die-cut border, "
        f"the sticker clearly shows the text: '{text}'"
    )

def add_speech_text_to_sticker_colab(image_path, speech_text, out_path=None):
    from PIL import Image, ImageDraw, ImageFont
    import re
    import os

    img = Image.open(image_path).convert("RGBA")
    W, H = img.size
    draw = ImageDraw.Draw(img)

    text = (speech_text or "").strip()
    if not text:
        return image_path

    # ✅ 字更大：原本 max(40, W//14) 太小
    font_size = max(84, W // 8)   # 你也可以改成 max(64, W//9) 比較保守

    def _wrap_lines(font):
        max_text_width = int(W * 0.86)  # 沒底板可以放寬一點
        # 用像素寬度換行（避免中文寬度不一致）
        t = re.sub(r"\s+", " ", text).strip()
        if not t:
            return []
        lines, buf = [], ""
        for ch in t:
            test = buf + ch
            w = draw.textbbox((0, 0), test, font=font)[2]
            if w <= max_text_width:
                buf = test
            else:
                if buf:
                    lines.append(buf)
                buf = ch
        if buf:
            lines.append(buf)
        return lines[:2]  # 最多兩行

    # ✅ 使用粗體中文字型（你前面已定義 FONT_PATH_BOLD）
    # 若你變數名不同，改成你的字型路徑即可
    font = ImageFont.truetype(FONT_PATH_BOLD, font_size)

    # 若兩行還是太長，最多自動縮小 2 次
    lines = _wrap_lines(font)
    for _ in range(2):
        if not lines:
            return image_path
        too_wide = False
        for line in lines:
            w = draw.textbbox((0, 0), line, font=font)[2]
            if w > int(W * 0.88):
                too_wide = True
                break
        if too_wide:
            font_size = int(font_size * 0.85)
            font = ImageFont.truetype(FONT_PATH_BOLD, font_size)
            lines = _wrap_lines(font)
        else:
            break

    if not lines:
        return image_path

    # 文字高度
    line_h = draw.textbbox((0, 0), "測", font=font)[3]
    gap = int(line_h * 0.18)
    pad_y = int(H * 0.04)

    # ✅ 放在底部，但不畫黑底
    total_h = line_h * len(lines) + gap * (len(lines) - 1)
    y0 = H - total_h - pad_y

    # ✅ 沒底板時描邊要更粗才清楚
    stroke = max(8, W // 90)

    y = y0
    for line in lines:
        w = draw.textbbox((0, 0), line, font=font)[2]
        x = (W - w) // 2
        draw.text(
            (x, y),
            line,
            font=font,
            fill=(255, 255, 255, 255),      # 白字
            stroke_width=stroke,
            stroke_fill=(0, 0, 0, 255),      # 黑描邊（更實）
        )
        y += line_h + gap

    # ✅ 強制輸出 PNG（避免 RGBA->JPEG 出錯）
    if out_path is None:
        base, _ = os.path.splitext(image_path)
        out_path = base + "_caption.png"

    root, ext = os.path.splitext(out_path)
    if ext.lower() in [".jpg", ".jpeg"]:
        out_path = root + ".png"

    img.save(out_path, format="PNG")
    return out_path

In [6]:
# =============================================================================
# 4. 核心功能區：工廠運作邏輯
# =============================================================================
# ===== 多維度情緒資料庫（從「語音轉貼圖_初步版本.ipynb」保留概念）=====
EMOTION_DATABASE = {
    "anger": {
        "zh": "憤怒",
        "description": "充滿怒火、不滿和激動",
        "visual": "眉頭緊皺、咬牙切齒、臉色漲紅",
        "effects": "火焰、閃電、爆炸特效",
        "colors": ["深紅色", "橙紅色", "暗灰色"],
        "mood": "強烈、激烈、暴躁",
        "intensity_high": "暴怒、憤慨、火冒三丈",
        "intensity_mid": "生氣、不悅、煩躁",
        "intensity_low": "微慍、不快、皺眉"
    },
    "disgust": {
        "zh": "厭惡",
        "description": "反感、排斥和不適",
        "visual": "皺鼻子、嘴角下垂、後退姿態",
        "effects": "臭味波紋、綠色煙霧、警告符號",
        "colors": ["暗綠色", "灰綠色", "土黃色"],
        "mood": "不適、排斥、嫌惡",
        "intensity_high": "作嘔、極度厭惡、噁心到爆",
        "intensity_mid": "厭惡、反感、嫌棄",
        "intensity_low": "不喜歡、皺眉、略嫌"
    },
    "fear": {
        "zh": "恐懼",
        "description": "緊張、害怕和不安",
        "visual": "瞳孔放大、冷汗、縮肩發抖",
        "effects": "陰影、顫抖線條、驚嚇符號",
        "colors": ["深紫色", "深藍色", "灰黑色"],
        "mood": "緊繃、驚慌、不安",
        "intensity_high": "驚恐、嚇壞、魂飛魄散",
        "intensity_mid": "害怕、緊張、心慌",
        "intensity_low": "不安、擔心、戒備"
    },
    "joy": {
        "zh": "喜悅",
        "description": "快樂、興奮和滿足",
        "visual": "燦爛笑容、眼睛彎彎、跳躍姿勢",
        "effects": "星星、彩帶、閃亮光點",
        "colors": ["明黃色", "粉紅色", "天藍色"],
        "mood": "歡樂、愉悅、雀躍",
        "intensity_high": "狂喜、欣喜若狂、樂開懷",
        "intensity_mid": "開心、快樂、滿足",
        "intensity_low": "微笑、愉快、心情好"
    },
    "sadness": {
        "zh": "悲傷",
        "description": "難過、失落和低落",
        "visual": "眼角含淚、嘴角下垂、垂頭喪氣",
        "effects": "雨滴、淚水、小水窪",
        "colors": ["淺藍色", "灰藍色", "淡紫色"],
        "mood": "憂鬱、失落、感傷",
        "intensity_high": "痛哭、心碎、悲痛欲絕",
        "intensity_mid": "難過、沮喪、失落",
        "intensity_low": "淡淡哀傷、悶悶的、低落"
    },
    "surprise": {
        "zh": "驚訝",
        "description": "意外、震驚和突然感",
        "visual": "張大嘴巴、眉毛上揚、眼睛睜大",
        "effects": "驚嘆號、爆炸線、亮光",
        "colors": ["亮橙色", "亮黃色", "白色"],
        "mood": "突發、驚奇、意外",
        "intensity_high": "震驚、驚呆、不可思議",
        "intensity_mid": "驚訝、意外、哇塞",
        "intensity_low": "小驚喜、略意外、哦？"
    },
    "neutral": {
        "zh": "平靜",
        "description": "冷靜、平和和中性",
        "visual": "表情自然、眼神平和、姿態放鬆",
        "effects": "簡約線條、幾何圖形",
        "colors": ["淡藍色", "米白色", "淺灰色"],
        "mood": "淡定、平穩、理性",
        "intensity_high": "非常冷靜、泰然自若、穩如泰山",
        "intensity_mid": "平靜、正常、淡定",
        "intensity_low": "略微無感、淡淡的、還好"
    }
}

EMOTION_FALLBACK = {
    "positive": "joy",
    "negative": "sadness",
    "POSITIVE": "joy",
    "NEGATIVE": "sadness",
    "NEUTRAL": "neutral"
}

def analyze_emotions_multi(text: str, top_k: int = 3):
    """把 pipeline 的 all_scores 轉成排序後的多維度情緒列表"""
    if not text or not emotion_classifier:
        return []

    results = emotion_classifier(text[:512])

    # pipeline 回傳格式通常是：[ [ {label, score}, ... ] ]
    if isinstance(results, list) and len(results) > 0 and isinstance(results[0], list):
        scores = results[0]
    else:
        scores = results

    cleaned = []
    for r in scores:
        label = r.get("label", "")
        score = float(r.get("score", 0))
        label = EMOTION_FALLBACK.get(label, label).lower()
        cleaned.append({"label": label, "score": score})

    cleaned.sort(key=lambda x: x["score"], reverse=True)
    return cleaned[:top_k]

def generate_emotion_description(emotions):
    """用 top1 + 次要情緒 組合成『組合描述』給 Gemini 用"""
    if not emotions:
        return "（未偵測到情緒）"

    primary = emotions[0]
    primary_label = EMOTION_FALLBACK.get(primary["label"], primary["label"])
    primary_score = primary["score"]

    emotion_data = EMOTION_DATABASE.get(primary_label, EMOTION_DATABASE["neutral"])

    # 強度分級（你可以自己調門檻）
    if primary_score >= 0.60:
        intensity = emotion_data["intensity_high"]
    elif primary_score >= 0.35:
        intensity = emotion_data["intensity_mid"]
    else:
        intensity = emotion_data["intensity_low"]

    # 次要情緒（簡單挑 score >= 0.20 的）
    secondary = []
    for e in emotions[1:]:
        if e["score"] >= 0.20:
            zh = EMOTION_DATABASE.get(e["label"], EMOTION_DATABASE["neutral"])["zh"]
            secondary.append(f"{zh}({e['score']:.0%})")

    desc = f"""【多維度情緒組合描述】
- 主要情緒：{emotion_data['zh']}（{primary_score:.0%}）→ {intensity}
- 情緒特徵：{emotion_data['description']}
- 視覺表現：{emotion_data['visual']}
- 特效元素：{emotion_data['effects']}
- 建議配色：{", ".join(emotion_data['colors'])}
- 氛圍感受：{emotion_data['mood']}
"""
    if secondary:
        desc += f"- 次要情緒：{', '.join(secondary)}\n"

    return desc

def generate_image_from_pollinations(prompt):
    """
    負責生圖的功能 (Pollinations AI)
    """
    try:
        encoded_prompt = quote(prompt)
        url = f"https://image.pollinations.ai/prompt/{encoded_prompt}"
        params = {
            "width": 1024,
            "height": 1024,
            "model": "flux",
            "seed": int(time.time()),
            "nologo": "true"
        }

        print(f"正在請求生圖... Prompt: {prompt[:30]}...")
        response = requests.get(url, params=params, timeout=60)

        if response.status_code == 200:
            filename = f"sticker_{int(time.time())}.jpg"
            with open(filename, "wb") as f:
                f.write(response.content)
            return filename
        else:
            print(f"生圖失敗，狀態碼: {response.status_code}")
            return None

    except Exception as e:
        print(f"生圖發生錯誤: {e}")
        return None

def process_pipeline(api_key, audio_path, use_gemini):
    """
    總指揮：錄音 -> 文字 -> 情緒 -> 提示詞 (Gemini 可選) -> 圖片
    """
    # --- 防呆檢查 ---
    # 只有在「使用 Gemini」時，才要求輸入 API Key
    if use_gemini and not api_key:
        return "❌ 已選擇使用 Gemini，請輸入 API Key", "無", "無", None

    # 不論是否使用 Gemini，語音都是必要的
    if not audio_path:
        return "❌ 請先錄音", "無", "無", None

    try:
        # ============================================================
        # 步驟 A: 語音轉文字
        # ============================================================
        result = whisper_model.transcribe(
            audio_path,
            language="zh",          # 明確指定中文
            task="transcribe",
            fp16=False,             # ❗CPU 一定要 False
            temperature=0.0,        # 降低隨機性，提高穩定度
            beam_size=5,            # Beam search 提升準確度
            best_of=5,
            patience=1.0,
            condition_on_previous_text=True,
            initial_prompt="以下是普通話的句子。",  # 中文語境提示
        )
        transcribed_text = result["text"]

        # ============================================================
        # 步驟 B: 多維度情緒分析 + 組合描述
        # ============================================================
        emotions = analyze_emotions_multi(transcribed_text, top_k=3)
        emotion_combo_desc = generate_emotion_description(emotions)

        # 讓系統仍保留「主要情緒 label」（給基本 prompt 用）
        detected_emotion = emotions[0]["label"] if emotions else "neutral"

        # ============================================================
        # 步驟 C: 產生生圖提示詞（Gemini 可選 / 可退回基本版本）
        # ============================================================
        prompt_for_image = None  # ✅ 真的拿去生圖的 prompt（要乾淨）
        prompt_for_ui = None     # ✅ 顯示給使用者看的 prompt（可加標籤）
        gemini_error = None      # ✅ Gemini 失敗原因（回傳給 UI）

        # ----------------------------
        # C-1. 使用者選擇用 Gemini → 嘗試優化提示詞
        # ----------------------------
        if use_gemini:
            try:
                # 1. 建立 Client（新版 SDK 寫法）
                client = genai.Client(api_key=api_key)

                # 2. 設定給 Gemini 的 Prompt（說清楚規則，避免亂輸出）
                prompt_text = f"""
You are an AI sticker designer.

Task:
Based on the speech content "{transcribed_text}",
generate ONE single English prompt for image generation.

Emotion reference (important):
{emotion_combo_desc}

Requirements:
1. Style: Cute 3D Pixar style sticker
2. Content: A chibi character with facial expression matching the emotion above
3. Keywords (must include): white background, high quality, vector style, die-cut border
4. Output format: ONE single line English prompt only
"""

                # 3. 發送請求給 Gemini
                response = client.models.generate_content(
                    model="gemini-2.5-flash",
                    contents=prompt_text
                )

                # 4. 取得 Gemini 回傳的提示詞
                prompt_for_image = (getattr(response, "text", "") or "").strip()
                prompt_for_ui = "[GEMINI PROMPT] " + prompt_for_image

                if prompt_for_image:
                    print("✅ 使用 Gemini 成功產生提示詞")
                else:
                    print("⚠️ Gemini 回傳為空，將改用基本提示詞")
                    prompt_for_image = None

            except Exception as e:
                # Gemini 可能遇到 429 / 503 / quota 問題
                gemini_error = f"{type(e).__name__}: {e}"
                print("⚠️ Gemini 暫時無法使用，將改用基本提示詞")
                print(str(e))
                prompt_for_image = None

        # ============================================================
        # C-2. 若「未使用 Gemini」或「Gemini 失敗」
        #     → 使用【基本提示詞】並提供給使用者查看
        # ============================================================
        if not prompt_for_image:
            # ✅ 生圖用：不要加 [BASIC PROMPT]，避免被模型當成要畫出來的文字
            prompt_for_image = (
                "Cute 3D Pixar style sticker, chibi character, "
                f"clear facial expression showing {detected_emotion}, "
                "simple and clean composition, "
                "white background, high quality, vector style, die-cut border"
            )

            # ✅ UI 顯示用：可以加標籤，讓使用者知道這是 basic 版本
            if gemini_error:
                prompt_for_ui = f"[GEMINI ERROR] {gemini_error}\n[BASIC PROMPT] {prompt_for_image}"
            else:
                prompt_for_ui = "[BASIC PROMPT] " + prompt_for_image

            print("✅ 使用基本提示詞（未使用 Gemini 或 Gemini 不可用）")

        # ============================================================
        # 步驟 D: 自動生圖
        # ============================================================
        image_path = generate_image_from_pollinations(prompt_for_image)

        # ✅ 生圖成功才貼字（只貼一次）
        if image_path:
            image_path = add_speech_text_to_sticker_colab(
                image_path=image_path,
                speech_text=transcribed_text
            )

        # ============================================================
        # 步驟 E: 回傳結果給 UI
        # ============================================================
        # UI 第2欄建議顯示「多維度情緒組合描述」會更有資訊量
        if image_path:
            return transcribed_text, emotion_combo_desc, prompt_for_ui, image_path
        else:
            return transcribed_text, emotion_combo_desc, prompt_for_ui, None

    except Exception as e:
        error_msg = f"❌ 流程發生錯誤: {str(e)}"
        print(error_msg)
        return error_msg, "錯誤", "錯誤", None



In [7]:
# =============================================================================
# 5. 介面設計區 (Gradio)
# =============================================================================
theme = gr.themes.Soft()

with gr.Blocks(theme=theme, title="AI 語音貼圖產生器 (New GenAI SDK)") as demo:

    gr.Markdown(
        """
        # 🎙️ AI 語音情緒貼圖產生器 (官方最新 SDK 版)
        ### 使用 Google 最新 GenAI SDK + 自動生圖
        """
    )

    with gr.Row():
        api_key_input = gr.Textbox(
            label="🔑 請輸入 Gemini API Key",
            type="password",
            placeholder="貼上你的 API Key...",
        )

    with gr.Row():
        # 左邊：操作區
        with gr.Column(scale=1):
            gr.Markdown("### 1. 輸入語音 🎤")
            audio_input = gr.Audio(sources=["microphone", "upload"], type="filepath")

            # ✅ 新增：是否使用 Gemini 優化提示詞
            use_gemini_checkbox = gr.Checkbox(
                label="🤖 使用 Gemini 優化提示詞（若不可用將自動退回基本版本）",
                value=True
            )

            btn_generate = gr.Button("✨ 開始生成 (文字+圖片)", variant="primary", size="lg")

            gr.Markdown("### 2. 分析結果 📊")
            text_output = gr.Textbox(label="語音轉文字", interactive=False)
            emotion_output = gr.Textbox(label="偵測情緒", interactive=False)
            prompt_output = gr.Textbox(label="生成的提示詞", interactive=False, lines=2)

        # 右邊：圖片展示區
        with gr.Column(scale=1):
            gr.Markdown("### 3. 生成的貼圖 🖼️")
            image_output = gr.Image(
                label="AI 生成結果",
                type="filepath",
                height=512
            )


    # 綁定功能
    btn_generate.click(
      fn=process_pipeline,
      inputs=[api_key_input, audio_input, use_gemini_checkbox],
      outputs=[text_output, emotion_output, prompt_output, image_output]
)



In [8]:
# =============================================================================
# 6. 啟動
# =============================================================================
print("🚀 介面啟動中！請點擊下方連結...")
demo.launch(debug=True)

🚀 介面啟動中！請點擊下方連結...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://39e16772dea5df4566.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


⚠️ Gemini 暫時無法使用，將改用基本提示詞
503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
✅ 使用基本提示詞（未使用 Gemini 或 Gemini 不可用）
正在請求生圖... Prompt: Cute 3D Pixar style sticker, c...
⚠️ Gemini 暫時無法使用，將改用基本提示詞
503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
✅ 使用基本提示詞（未使用 Gemini 或 Gemini 不可用）
正在請求生圖... Prompt: Cute 3D Pixar style sticker, c...
✅ 使用 Gemini 成功產生提示詞
正在請求生圖... Prompt: A cute 3D Pixar style chibi ch...
⚠️ Gemini 暫時無法使用，將改用基本提示詞
503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
✅ 使用基本提示詞（未使用 Gemini 或 Gemini 不可用）
正在請求生圖... Prompt: Cute 3D Pixar style sticker, c...
✅ 使用 Gemini 成功產生提示詞
正在請求生圖... Prompt: Cute 3D Pixar style sticker of...
⚠️ Gemini 暫時無法使用，將改用基本提示詞
503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.